<h1>Reinforcement Learning</h1>
<h6>Saket Tiwari</h6>
Date: 08 July 2019

In [1]:
#isme humlog machine ko sikhaynge ki kab kon sa action perform karna
#reinforcement me humlog model ko btaynge ki action jo liya h wo  sahi h ya galat humlog ko output nhi pata, reward btaynge


#state t par koi action perform kiye to wo state (t+1) me jayga.. lekin jo reward milega humlogon ko wo (t+1) ka reward milega

# alpha -  learning rate
# gamma - discount factor
# epsilon - exploration rate

#agar game me(5x4) 20 pos honge to q table me rows 20 hoga
#Q-learning -> Q-table -> rows- describes states, columns-action
#formula ->  Q(s_t , a_t)= r_(t+1) +   gamma* r_(t+2) + gamma^2 * r_(T+3)
#future me jo reward milne wala h usko hmlog disregard karte jaynge kyuki hmlog future me confirm nhi h
# ..ki wo reward milega ki nahi humlogon ko
#we are concerned with overall reward not the immediate reward

#Q(s_t , a_t)= r_(t+1) +   gamma* r_(t+2) + gamma^2 * r_(T+3)
# Q(s_(t+1), a_(t+1)) = r_(t+2) +gamma*r_(t+3) + gamma^2 *r_(t+4)

#Therefor final reward = Q(s_t , a_t) = r_(t+1) + gamma *Q(s_(t+1), a_(t+1))
# or  Q(s_t , a_t) = r_(t+1) +gamma * Q_max_(s_(t+1))   Q_max me  future me jitna reward milte wala h

#exploration rate explores the other states for highest reward

In [12]:
import gym
import random
import numpy as np
import time
from IPython.display import clear_output

In [3]:
# frozen ice -> slippery

In [4]:
from gym.envs.registration import register

In [5]:
#https://github.com/openai/gym/blob/master/gym/envs/__init__.py par ja kr randomness off krnge

try:
    register(
        id='FrozenLakeNoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery':False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )
except:
    pass

In [6]:
env_name = 'FrozenLakeNoSlip-v0'
env= gym.make(env_name)

In [7]:
env.observation_space

Discrete(16)

In [9]:
type(env.action_space)

gym.spaces.discrete.Discrete

In [24]:
class Agent():
    
    def __init__(self, env):
        
        self.isDiscrete =( type( env.action_space)) ==gym.spaces.discrete.Discrete
        
    def get_action(self,state):#check kareha ki discrete ya continuous
        
        if self.isDiscrete:
            action = random.choice(range(env.action_space.n)) 
        else:
            action = np.random.uniform( env.action_space.low,
                                      env.action_space.high,
                                      env.action_space.shape) #because it is a continuous h
        return action

    
agent= Agent(env)

In [26]:
total_reward=0
for ep in range(10):
    
    state= env.reset() # game fresh start hoga
    done=False # gave abhi over nahi hua h
    
    while not done:
        
        action= agent.get_action(state)
        state, reward, done, info = env.step(action)
        env.render()
        time.sleep(0.05)
        clear_output(wait=True)
env.close()

  (Up)
SFFF
FHFH
FFFH
HFFG


# Q-Learning

In [58]:
class Agent():
    
    def __init__(self, env, discountedRate=0.97, learningRate=0.01):
        
        self.isDiscrete =( type( env.action_space)) ==gym.spaces.discrete.Discrete
        self.stateSize=env.observation_space.n
        self.actionSize = env.action_space.n
        self.explorationRate=1.0
        self.discountedRate= discountedRate
        self.learningRate= learningRate
        self.qTable= 1e-4*np.random.random([self.stateSize, self.actionSize])
        
    def get_action(self,state):#check kareha ki discrete ya continuous
        
        qState= self.qTable[state] # we will get a single row with corresponding state #sbse pehle kaun se state pr h
        actionGreedy=np.argmax(qState)     
        actionRandom=env.action_space.sample()
        
        return actionRandom if random.random()<self.explorationRate else actionGreedy
    
    def train(self, state, action, next_state , reward, done): #(state, action, next_state , reward, done) is called experience
        qStateNext=self.qTable[next_state]
        if done:
            qStateNext = np.zeros([self.actionSize])
        else:
            qStateNext= self.qTable[next_state]
        qTarget = reward + self.discountedRate*np.max(qStateNext)
        qUpdate = qTarget - self.qTable[state, action]
        self.qTable[state, action]+=self.learningRate*qUpdate
        
        if done:
            self.explorationRate*=0.99
        
agent= Agent(env)

In [59]:
total_reward=0
for ep in range(100):  
    state= env.reset() # game fresh start hoga
    done=False # gave abhi over nahi hua h
    
    while not done:
        action= agent.get_action(state)
        nextState, reward, done, info = env.step(action)
        agent.train(state,action, nextState,reward,done)
        state=nextState
        total_reward+=reward
        print(f"s:{state},a:{action}")
        print(f'Ep:{ep},Goals:{total_reward}, Explore:{agent.explorationRate}')
        env.render()
        print(agent.qTable)
        time.sleep(0.05)
        clear_output(wait=True)
env.close()

s:15,a:2
Ep:99,Goals:10.0, Explore:0.36603234127322926
  (Right)
SFFF
FHFH
FFFH
HFFG
[[8.38348880e-05 8.50280486e-05 4.32886961e-05 4.42219907e-05]
 [6.16248146e-05 1.18628906e-05 7.60024896e-05 6.51180528e-05]
 [4.49707679e-05 8.35571330e-05 6.88482052e-05 6.91467659e-05]
 [4.92823556e-05 2.77991778e-05 9.28140945e-05 1.45866569e-05]
 [8.60915995e-05 8.31077326e-05 6.30356860e-05 2.76906476e-05]
 [5.24200642e-05 7.99565469e-06 1.45887604e-05 8.36816923e-06]
 [5.36613061e-05 4.59071319e-05 7.77647646e-05 1.16657799e-05]
 [6.52489991e-06 6.40866490e-05 4.63366484e-06 3.68075341e-05]
 [2.94979113e-05 6.83794444e-05 9.25961514e-05 4.50018394e-05]
 [4.37225205e-05 9.20260749e-05 2.61571505e-04 9.26948989e-05]
 [6.00667802e-05 3.99719271e-03 8.61110515e-05 8.38828547e-05]
 [7.53533889e-05 6.23302883e-05 1.81477396e-05 5.47302525e-05]
 [8.07319327e-05 2.16822646e-05 8.56134600e-05 1.20086852e-05]
 [5.00770479e-05 1.60978761e-07 1.19930572e-03 1.83869439e-06]
 [9.53734872e-06 9.91226337e-04 9

# Q-Learning with Neural Networks

In [37]:
import tensorflow as tf

In [60]:
class Agent():
    
    def __init__(self, env, discountedRate=0.97, learningRate=0.01):
        
        self.isDiscrete =( type( env.action_space)) ==gym.spaces.discrete.Discrete
        self.stateSize=env.observation_space.n
        self.actionSize = env.action_space.n
        
        self.explorationRate=1.0
        self.discountedRate= discountedRate
        self.learningRate= learningRate
        
        tf.reset_default_graph()
        #creating variables
        self.stateIn =tf.placeholder(tf.int32,shape=[1])
        self.actionIn =tf.placeholder(tf.int32,shape=[1])
        self.targetIn =tf.placeholder(tf.float32,shape=[1])
        
        self.state =tf.one_hot(self.stateIn, depth=self.stateSize)
        self.action =tf.one_hot(self.actionIn, depth=self.actionSize)
        self.qState= tf.layers.dense(self.state, units= env.action_space.n, name='Qtable')
        self.qAction = tf.reduce_sum(tf.multiply(self.qState, self.action), axis=1) #Scalar
        
        self.loss=tf.reduce_sum(tf.square(self.targetIn-self.qAction))
        self.optimizer=tf.train.AdamOptimizer(self.learningRate).minimize(self.loss)
        self.sess=tf.Session()
        self.sess.run(tf.global_variables_initializer())
        
    def get_action(self,state):#check kareha ki discrete ya continuous
        
        qState= self.sess.run(self.qState,feed_dict= {self.stateIn:[state]}) # we will get a single row with corresponding state #sbse pehle kaun se state pr h
        actionGreedy=np.argmax(qState)     
        actionRandom=env.action_space.sample()
        
        return actionRandom if random.random()<self.explorationRate else actionGreedy
    
    def train(self, state, action, next_state , reward, done): #(state, action, next_state , reward, done) is called experience
        
        if done:
            qStateNext = np.zeros([self.actionSize])
        else:
            qStateNext= self.sess.run(self.qState, feed_dict= {self.stateIn:[next_state] })
        qTarget = reward + self.discountedRate*np.max(qStateNext)
        
        
        feed ={ self.stateIn:[state],self.actionIn:[action], self.targetIn:[qTarget]}
        self.sess.run(self.optimizer, feed_dict=feed)
        if done:
            self.explorationRate*=0.99
    
    def __del__(self):
        self.sess.close()
        
agent= Agent(env)

In [62]:
total_reward=0
for ep in range(100):  
    state= env.reset() # game fresh start hoga
    done=False # gave abhi over nahi hua h
    
    while not done:
        action= agent.get_action(state)
        nextState, reward, done, info = env.step(action)
        agent.train(state,action, nextState,reward,done)
        state=nextState
        total_reward+=reward
        print(f"s:{state},a:{action}")
        print(f'Ep:{ep},Goals:{total_reward}, Explore:{agent.explorationRate}')
        env.render()
        
        with tf.variable_scope('Qtable', reuse=True): #if reuse=True weights will not reset.
            weights=agent.sess.run(tf.get_variable('kernel'))
        print(weights)
        time.sleep(0.05)
        clear_output(wait=True)
env.close()

s:15,a:2
Ep:99,Goals:60.0, Explore:0.13397967485796175
  (Right)
SFFF
FHFH
FFFH
HFFG
[[ 3.0034229e-01  1.5514053e-01  9.9149972e-02  1.4712311e-03]
 [ 2.9294816e-01 -5.4990214e-01  1.2902808e-01  1.3375770e-01]
 [ 3.4233361e-01  3.5908777e-01  5.2457903e-02  1.6361105e-01]
 [ 3.2497284e-01 -3.1679502e-01 -5.6305647e-02 -3.0368692e-03]
 [ 5.7104278e-02  1.4110394e-01 -6.3415742e-01  8.9487545e-02]
 [ 6.9905162e-02  1.0132694e-01 -1.0720506e-01 -7.2537959e-02]
 [-5.9418625e-01  3.7649924e-01 -7.7506649e-01  1.7263173e-01]
 [-9.5067620e-03 -2.0048982e-01 -4.7103602e-01 -1.1449993e-01]
 [ 1.5990813e-01 -3.8714001e-01 -1.7517290e-04  4.4217922e-02]
 [ 1.5699953e-02  3.8959095e-01 -3.4888476e-02 -5.6483757e-01]
 [ 4.2141238e-01  4.1830918e-01 -4.9695361e-01  1.9285630e-01]
 [-1.0826334e-01 -2.2063705e-01  2.0794761e-01 -1.6201201e-01]
 [ 4.8506117e-01  1.9182444e-01  4.3048787e-01  1.8552858e-01]
 [-6.5426648e-01  1.4091173e-01  2.1401879e-01  1.7491673e-01]
 [ 4.3396807e-01  4.3527949e-01  